In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import networkx.generators.random_graphs as ggen
from networkx.convert_matrix import to_numpy_matrix as g2a
from networkx.convert_matrix import from_numpy_matrix as a2g
from nxpd import draw

import dit

import itertools
import numpy as np
from numpy import array as npa

import random

In [3]:
def genAllComb(n):
    return list(itertools.product('01', repeat=n))

def genTranTable(comb):
    nComb = comb.shape[0]
    p = np.ones(nComb)/nComb
    iComb_ = np.random.choice(range(nComb), nComb, replace=True, p=p)
    comb_ = comb[iComb_]
    return comb_

def npa2str(a):
    return [''.join(x) for x in a.astype(str)]

def conditional_mutual_information(d, X, Y, Z):
    return dit.shannon.mutual_information(d, X, Y+Z) - dit.shannon.mutual_information(d, X, Z)

def genSysEnvVarInd(Yt, n):
    Et = list(set(range(n)) - set(Yt))
    Yt_ = [x+n for x in Yt]
    Et_ = [x+n for x in Et]
    return Yt, Et, Yt_, Et_

def NTIC(d, Yt):
    n = len(d.rvs) // 2
    Et = list(set(range(n)) - set(Yt))
    Yt_ = [x+n for x in Yt]
    Et_ = [x+n for x in Et]
    return dit.shannon.mutual_information(d, Yt_, Yt) - conditional_mutual_information(d, Yt_, Yt, Et)

def genAllSubsets(X):
    subsets = []
    for iLen in range(len(X)):
        subset = list(itertools.combinations(X, iLen+1))
        subsets = subsets + subset
    return subsets

In [4]:
n = 3
comb = npa(genAllComb(n))
comb_ = genTranTable(comb)
# tranTable = np.hstack([comb, comb_])
tranTable = np.hstack([comb, comb, comb_, comb_])
tranTable = npa2str(tranTable)

# tern into dist
prob = np.ones(len(tranTable))/len(tranTable)
d = dit.Distribution(tranTable, prob)
d.set_rv_names(range(len(d.rvs)))
d.alphabet = (('0', '1'),)*len(d.rvs)
print(d)

Class:          Distribution
Alphabet:       ('0', '1') for all rvs
Base:           linear
Outcome Class:  str
Outcome Length: 12
RV Names:       (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)

x              p(x)
000000001001   0.125
001001101101   0.125
010010001001   0.125
011011001001   0.125
100100000000   0.125
101101010010   0.125
110110111111   0.125
111111100100   0.125


In [5]:
X = [3]
Y = [2]
Z = [0, 1]
conditional_mutual_information(d, X, Y, Z)
NTIC(d, [0,1,2,3,4])

0.6556390622295662

In [ ]:
Yt = [0]

h_Yt = []
h_Et = []
h_NTIC = []

Yt, Et, Yt_, Et_ = genSysEnvVarInd(Yt, len(d.rvs)//2)
h_Yt.append(Yt)
h_Et.append(Et)
h_NTIC.append(NTIC(d, Yt))

# recruit 
Esubsets = genAllSubsets(Et)
NTIC_ = [NTIC(d, Yt + list(iSubset)) for iSubset in Esubsets]
NTIC_ = npa(NTIC_)
i_NTIC_max = np.argmax(NTIC_)
Yt_NTIC_max = Yt + list(Esubsets[i_NTIC_max])
Yt_NTIC_max

[0, 1, 2]

In [164]:
Yt_NTIC_max

[0, 1, 2]